# **IMPORT**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.functional as F

import json
import os
import torch

os.chdir('F:\\UNIVERSITY\\UNIVERSITY_DOCUMENTS\\CS231\\doan_v2')

from skimage import io, transform
from tqdm import tqdm
from torch import nn
from torch import optim

from sklearn.metrics import jaccard_score, hamming_loss, precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain, MultiOutputClassifier, MultiOutputRegressor 
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from torch.utils.data import Dataset, DataLoader
from torch.nn import Linear, ReLU, Sigmoid, Softmax, Dropout, Sequential

from torchvision import transforms, utils
from torchvision.transforms import ToTensor
from torchvision.models import vgg16, resnet50, densenet169
from torchvision.models.vgg import VGG16_Weights

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **FUNCTION**

In [2]:
# Load json
def load_json(path):
    with open(path, 'r', encoding='utf-8') as file:
        film_dic = json.load(file)
        return film_dic

# **DATASET**

In [3]:
classes = [
    'action', 'adventure', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'horror', 'mystery', 'thriller', 'romance', 'scifi', 'others'
]

config = {
    'img_size': (224, 224),
    'epochs': 10,
    'batches': 10,
}

In [4]:
class MovieGenreDataset():
    def __init__(
            self, 
            annot_path='data/train_new.json',
            features_dir='features_VGG',
            classes=[
                'action', 'adventure', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'horror', 'mystery', 'thriller', 'romance', 'scifi', 'others'
            ],
        ) -> None:
        annotation = load_json(annot_path)
        set_name = os.path.basename(annot_path).split('.')[0]
        
        # Load X
        npy_features_name = os.listdir(features_dir)
        npy_file_names = [f'{id}.npy' for id in annotation.keys() if f'{id}.npy' in npy_features_name]
        npy_file_paths = [os.path.join(features_dir, npy_file_name) for npy_file_name in npy_file_names ]

        self.X = np.array([
            np.load(npy_file_path) 
            for npy_file_path in tqdm(npy_file_paths, desc=f"Loading X_{set_name}")
        ])
        
        # Load y
        mlb = MultiLabelBinarizer()
        mlb.fit([classes])

        genres = [
            val['genre'] 
            for val in tqdm(annotation.values(), desc="Loading y")
        ]
        self.y = mlb.transform(genres)

    def get_data(self):
        return self.X, self.y
        

## *DENSENET*

In [5]:
train_Densenet169_dataset = MovieGenreDataset(
    annot_path='data/train_new.json',
    features_dir='features_Densenet169',
)

val_Densenet169_dataset = MovieGenreDataset(
    annot_path='data/val.json',
    features_dir='features_Densenet169',
)

test_Densenet169_dataset = MovieGenreDataset(
    annot_path='data/test.json',
    features_dir='features_Densenet169',
)

Loading y: 100%|██████████| 2513/2513 [00:00<00:00, 627883.84it/s]


In [6]:
X_train_Densenet, y_train_Densenet = train_Densenet169_dataset.get_data()
X_val_Densenet, y_val_Densenet = val_Densenet169_dataset.get_data()
X_test_Densenet, y_test_Densenet = test_Densenet169_dataset.get_data()

# **PCA**

## *DENSENET*

In [7]:
pca_Densenet = PCA(n_components=0.4)
X_train_Densenet_new = pca_Densenet.fit_transform(X_train_Densenet)
X_val_Densenet_new = pca_Densenet.transform(X_val_Densenet)
X_test_Densenet_new = pca_Densenet.transform(X_test_Densenet)

# **MODELING**

## *MultiOutputClassifier*

In [10]:
# Random Forest
class MultiOutputRegressorRandomForest():
    def __init__(self):
        self.model = None
        self.model_tuning = None
        self.params = {
            'estimator__bootstrap': [True, False],
            'estimator__max_depth': [30, 50, 70, 90, None],
            'estimator__min_samples_leaf': [1, 2, 4],
            'estimator__min_samples_split': [2, 5, 10],
            'estimator__n_estimators': [50, 100],
        }

    def fit(self, X, y):
        print('Fitting model')
        self.model = MultiOutputRegressor(RandomForestRegressor())
        self.model.fit(X, y)

    def fit_tuning(self, X, y):
        print('Fitting tuning model')
        estimator = MultiOutputRegressor(RandomForestRegressor())
        self.model_tuning = RandomizedSearchCV(
            estimator=estimator, 
            param_distributions=self.params, 
            random_state=0,
            n_iter=2,
            verbose=True,
            n_jobs=-1,
        )
        self.model_tuning.fit(X, y)

    def predict(self, X, mode='default'):
        print('Prediction')
        if mode=='default' and self.model != None:
            prediction = self.model.predict(X)
        elif mode=='tune' and self.model_tuning != None:
            prediction = self.model_tuning.predict(X)
        else:
            print('Cant predict the X set !!')
            prediction = np.array([])
        return prediction


In [52]:
# Random Forest
class MultiOutputClassifierLogisticRegression():
    def __init__(self):
        self.model = None
        self.model_tuning = None

    def fit(self, X, y):
        print('Fitting model')
        self.model = MultiOutputClassifier(LogisticRegression())
        self.model.fit(X, y)

    def predict(self, X, mode='default'):
        print('Prediction')
        if mode=='default' and self.model != None:
            prediction = self.model.predict(X)
        elif mode=='tune' and self.model_tuning != None:
            prediction = self.model_tuning.predict(X)
        else:
            print('Cant predict the X set !!')
            prediction = np.array([])
        return prediction


### *No Tuning*

#### *Random Forest*

In [58]:
# Densenet169
rf_dense = MultiOutputRegressorRandomForest()
rf_dense.fit(X_train_Densenet_new, y_train_Densenet)

Fitting model


In [59]:
# Predict
prediction_rf_dense_test = rf_dense.predict(X_test_Densenet_new)
prediction_rf_dense_val = rf_dense.predict(X_val_Densenet_new)
np.save('evaluation/prediction_rf_dense_test.npy', prediction_rf_dense_test)
np.save('evaluation/prediction_rf_dense_val.npy', prediction_rf_dense_val)

Prediction
Prediction


#### *Logistic Regression*

In [60]:
# Densenet169
logistic_r_dense = MultiOutputClassifierLogisticRegression()
logistic_r_dense.fit(X_train_Densenet_new, y_train_Densenet)

Fitting model


In [61]:
# Predict
prediction_logistic_r_dense_test = logistic_r_dense.predict(X_test_Densenet_new)
prediction_logistic_r_dense_val = logistic_r_dense.predict(X_val_Densenet_new)
np.save('evaluation/prediction_logistic_r_dense_test.npy', prediction_logistic_r_dense_test)
np.save('evaluation/prediction_logistic_r_dense_val.npy', prediction_logistic_r_dense_val)

Prediction
Prediction


#### *SVC*

In [11]:
# Random Forest
from sklearn.svm import SVC
class MultiOutputClassifierSVC():
    def __init__(self):
        self.model = None
        self.model_tuning = None

    def fit(self, X, y):
        print('Fitting model')
        self.model = MultiOutputClassifier(SVC())
        self.model.fit(X, y)

    def predict(self, X, mode='default'):
        print('Prediction')
        if mode=='default' and self.model != None:
            prediction = self.model.predict(X)
        elif mode=='tune' and self.model_tuning != None:
            prediction = self.model_tuning.predict(X)
        else:
            print('Cant predict the X set !!')
            prediction = np.array([])
        return prediction


In [12]:
svc_dense = MultiOutputClassifierSVC()
svc_dense.fit(X_train_Densenet_new, y_train_Densenet)

Fitting model


In [13]:
# # Predict
prediction_svc_dense_test = svc_dense.predict(X_test_Densenet_new)
prediction_svc_dense_val = svc_dense.predict(X_val_Densenet_new)
np.save('evaluation/prediction_svc_dense_test.npy', prediction_svc_dense_test)
np.save('evaluation/prediction_svc_dense_val.npy', prediction_svc_dense_val)

Prediction
Prediction
